# During routine clinical rounding, 

Respiratory Therapy students are expected to have an indepth understanding of what is occurring with Oxygenation and Ventilation in relation to the current artieral blood gas. While there are a plethora of other indices and metrics that are available to a Respiratory Therapist (e.g. From Mechanical Power to Ventilatory Ratio), a large majority of clinical discussions center around the ABG due to the core components of Ventilation and Oxygenation being centered around so many physiological processes (e.g. Diabetic Ketoacidosis vs. Sepsis with Lactic Acidosis vs. Respiratory Complications from Opiate Overdose).

Please proceed by pressing Shift+Enter, Twice.

In [34]:
import tkinter as tk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.colors as mcolors

def calculate_pH(PaCO2, HCO3):
    pK = 6.1
    PCO2_conversion = 0.03
    pH = pK + np.log10(HCO3 / (PCO2_conversion * PaCO2))
    return pH

def classify_abg(pH, PaCO2, HCO3):
    if pH < 7.35:
        if PaCO2 > 45:
            if HCO3 > 26:
                return "Partially Compensated Respiratory Acidosis", 'Yellow'
            elif HCO3 < 22:
                return "Mixed Acidosis", 'Red'
            else:
                return "Uncompensated Respiratory Acidosis", 'Orange'
        elif HCO3 < 22:
            if PaCO2 < 35:
                return "Partially Compensated Metabolic Acidosis", 'Yellow'
            elif PaCO2 > 45:
                return "Mixed Acidosis", 'Red'
            else:
                return "Uncompensated Metabolic Acidosis", 'Orange'
        else:
            return "Undefined", 'gray'
    elif pH > 7.45:
        if PaCO2 < 35:
            if HCO3 < 22:
                return "Partially Compensated Respiratory Alkalosis", 'cyan'
            elif HCO3 > 26:
                return "Mixed Alkalosis", 'Purple'
            else:
                return "Uncompensated Respiratory Alkalosis", 'blue'
        elif HCO3 > 26:
            if PaCO2 > 45:
                return "Partially Compensated Metabolic Alkalosis", 'cyan'
            elif PaCO2 < 35:
                return "Mixed Alkalosis", 'Purple'
            else:
                return "Uncompensated Metabolic Alkalosis", 'Blue'
        else:
            return "Undefined", 'gray'
    else:
        if 35 <= PaCO2 <= 45 and 22 <= HCO3 <= 26:
            return "Normal", 'green'
        else:
            return "Undefined", 'gray'

def calculate_possible_PaCO2_HCO3(pH, PaCO2, HCO3, radius=2, num_points=100):
    pK = 6.1
    PCO2_conversion = 0.03

    angles = np.linspace(0, 2 * np.pi, num_points)
    dHCO3_values = np.zeros(num_points)
    dPaCO2_values = np.zeros(num_points)

    for i, angle in enumerate(angles):
        dHCO3 = radius * np.sin(angle)
        dPaCO2 = radius * np.cos(angle)

        dHCO3_values[i] = HCO3 + dHCO3
        dPaCO2_values[i] = PaCO2 + dPaCO2

    pH_values = pK + np.log10(dHCO3_values / (PCO2_conversion * dPaCO2_values))

    return pH_values, dHCO3_values

def plot_PaCO2_lines(ax):
    pK = 6.1
    PCO2_conversion = 0.03

    for PaCO2 in range(10, 110, 10):
        HCO3_values = np.linspace(5, 50, 100)
        pH_values = pK + np.log10(HCO3_values / (PaCO2 * PCO2_conversion))
        ax.plot(pH_values, HCO3_values, 'k', alpha=0.5)
        ax.text(pH_values[-1], HCO3_values[-1], f'{PaCO2}', fontsize=8, verticalalignment='bottom')

def interactive_plot(PaCO2, HCO3):
    pH = calculate_pH(PaCO2, HCO3)
    
    classification, color = classify_abg(pH, PaCO2, HCO3)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlabel("pH")
    ax.set_ylabel("HCO3")
    
    # Calculate pH for the current PaCO2 and HCO3 values
    pH = calculate_pH(PaCO2, HCO3)

    # Plot the point
    ax.plot(pH, HCO3, 'ro', markersize=10)

    # Plot the circle
    pH_values, HCO3_values = calculate_possible_PaCO2_HCO3(pH, PaCO2, HCO3, radius=2)  # You can change the radius to modify the size of the circle
    ax.plot(pH_values, HCO3_values, 'r-', linewidth=2)

    # Plot the color map
    pH_grid, HCO3_grid = np.meshgrid(np.linspace(6.2, 8.4, 200), np.linspace(5, 50, 200))
    pK = 6.1
    PCO2_conversion = 0.03
    PaCO2_grid = HCO3_grid / (10 ** (pH_grid - pK) * PCO2_conversion)

    color_map = np.empty((200, 200), dtype=object)
    for i in range(200):
        for j in range(200):
            color_map[i, j] = classify_abg(pH_grid[i, j], PaCO2_grid[i, j], HCO3_grid[i, j])[1]

    # Convert color_map to RGBA
    color_rgba = np.empty((200, 200, 4), dtype=np.float32)
    for i, color_name in enumerate(np.unique(color_map)):
        color_rgba[color_map == color_name] = mcolors.to_rgba(color_name)

    ax.imshow(color_rgba, extent=[6.2, 8.4, 5, 50], origin='lower', alpha=0.6, aspect='auto')

    # Add PaCO2 lines
    plot_PaCO2_lines(ax)

    # Get classification for the point and display it on the plot
    classification, _ = classify_abg(pH, PaCO2, HCO3)
    plt.text(0.05, 0.95, f'Classification: {classification}', horizontalalignment='left',
             verticalalignment='center', transform=ax.transAxes, bbox=dict(facecolor='red', alpha=0.5))

    plt.show()
    
# Create sliders
PaCO2_slider = widgets.FloatSlider(min=10, max=100, step=1, value=40, description='PaCO2')
HCO3_slider = widgets.FloatSlider(min=5, max=50, step=1, value=25, description='HCO3')

# Display the sliders and the plot
widgets.interact(interactive_plot, PaCO2=PaCO2_slider, HCO3=HCO3_slider)


interactive(children=(FloatSlider(value=40.0, description='PaCO2', min=10.0, step=1.0), FloatSlider(value=25.0…

<function __main__.interactive_plot(PaCO2, HCO3)>

The introduction of Arterial Blood Gases is now interactive from the start, as opposed to being static and repetative from the beginning.

# The Problem

Multivariate Calculus and Differential Equations: 

Yea, I said Calculus.

The first problem arises from the physiological relationships among ABG variables. PaCO2 and HCO3- are connected through the Henderson-Hasselbalch equation, which is used to calculate blood pH. 

## This equation in itself introduces logarithmic functions into the equation, a level of complexity above simple linear relationships. 

Furthermore, understanding how changes in one variable affect another over time (rate of change) leads us into the realm of derivatives, 

## which is a central concept in calculus.

The Henderson-Hasselbalch equation is given by:

$$ \text{pH} = \text{p}K_a + \log_{10}\left(\frac{[\text{A}^-]}{[\text{HA}]}\right) $$

For the bicarbonate system in blood, it can be rewritten as:

$$ \text{pH} = 6.1 + \log_{10}\left(\frac{[\text{HCO}_3^-]}{0.03 \times \text{PaCO}_2}\right) $$


To add complication to this equation,

We also have to consider the classification of acid-base disorders.

# The Classification System:

## Normal: 

This occurs when the pH, PaCO2, and HCO3 are all within the normal range. 

### pH: 7.35-7.45 
### PaCO2: 35-45 mmHg 
### HCO3: 22-26 mEq/L 

Everything is balanced in the body.

## Uncompensated Respiratory Acidosis: 

This means that the pH is lower than normal (acidic), and the primary issue is related to breathing (high PaCO2), with the body not yet compensating.

## Partially Compensated Respiratory Acidosis: 

Here, the pH is still lower than normal and PaCO2 is high, but the body has started to compensate by increasing the HCO3 level.

## Mixed Acidosis: 

This occurs when the pH is low, but both the respiratory (high PaCO2) and metabolic (low HCO3) systems are contributing to the acidic condition.

## Uncompensated Metabolic Acidosis: 

Here, the pH is lower than normal due to metabolic issues (low HCO3), and the body has not started compensating yet.

## Partially Compensated Metabolic Acidosis: 

In this state, the pH is still lower than normal, the HCO3 is low, but the body has started to compensate by decreasing the PaCO2 level.

## Uncompensated Respiratory Alkalosis: 

This means that the pH is higher than normal (alkaline), and the primary issue is related to breathing (low PaCO2), with the body not compensating yet.

## Partially Compensated Respiratory Alkalosis: 

Here, the pH is still higher than normal and PaCO2 is low, but the body has started to compensate by decreasing the HCO3 level.

## Mixed Alkalosis: 

This occurs when the pH is high, but both the respiratory (low PaCO2) and metabolic (high HCO3) systems are contributing to the alkaline condition.

## Uncompensated Metabolic Alkalosis: 

Here, the pH is higher than normal due to metabolic issues (high HCO3), and the body has not started compensating yet.

## Partially Compensated Metabolic Alkalosis: 

In this state, the pH is still higher than normal, the HCO3 is high, but the body has started to compensate by increasing the PaCO2 level.

## Undefined: 

This condition refers to any result that doesn't fit into any of the above categories.

***Warning***

The code below is experimental and an attempt at recreating the above graph in a 3D representation where the rotation can be controlled to see how the relationship may look in an alternative view.

Future experiments with complex numbers and regraphing pose interesting questions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
def pH(HCO3, PCO2):
    return 6.1 + np.log10(HCO3/(0.03*PCO2))

In [ ]:
# Generate HCO3 and PCO2 values
HCO3 = np.linspace(10, 40, 100)  # typical range of HCO3- in arterial blood is 22-28 mEq/L
PCO2 = np.linspace(20, 60, 100)  # typical range of PCO2 in arterial blood is 35-45 mmHg

# Create a grid of (HCO3, PCO2) pairs
HCO3, PCO2 = np.meshgrid(HCO3, PCO2)


In [ ]:
# Calculate pH values for the grid
Z = pH(HCO3, PCO2)


In [ ]:
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(HCO3, PCO2, Z)

ax.set_xlabel('HCO3- (mEq/L)')
ax.set_ylabel('PCO2 (mmHg)')
ax.set_zlabel('pH')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive

def pH(HCO3, PCO2):
    return 6.1 + np.log10(HCO3/(0.03*PCO2))

def plot_func(HCO3_min, HCO3_max, PCO2_min, PCO2_max, elev, azim):
    HCO3 = np.linspace(HCO3_min, HCO3_max, 100)
    PCO2 = np.linspace(PCO2_min, PCO2_max, 100)
    HCO3, PCO2 = np.meshgrid(HCO3, PCO2)
    Z = pH(HCO3, PCO2)
    
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=elev, azim=azim)
    ax.plot_surface(HCO3, PCO2, Z, cmap='viridis')
    
    ax.set_xlabel('HCO3- (mEq/L)')
    ax.set_ylabel('PCO2 (mmHg)')
    ax.set_zlabel('pH')

interactive_plot = interactive(plot_func, 
                               HCO3_min=(10.0, 30.0), 
                               HCO3_max=(35.0, 60.0),
                               PCO2_min=(10.0, 30.0),
                               PCO2_max=(35.0, 60.0),
                               elev=(-90, 90),
                               azim=(-90, 90))
interactive_plot


Pretty cool when the Elev inverts with the Azim. If you want to break the graph, consider replacing the minimum values in the code with negative values. You can invert the equation and see how it still makes sense when inverted.

Math rewards you when you respect it's symmytries.

Forgive the misspellings.